In [3]:
import numpy as np
import pandas as pd

In [7]:
conf = pd.read_csv("ucdp-prio-acd-181.csv")
conf.head()

,conflict_id,location,side_a,side_a_id,side_a_2nd,side_b,side_b_id,side_b_2nd,incompatibility,territory_name,...,ep_end,ep_end_date,ep_end_prec,gwno_a,gwno_a_2nd,gwno_b,gwno_b.1,gwno_loc,region,version
0,200,Bolivia,Government of Bolivia,23,NaN,Popular Revolutionary Movement,719,NaN,2,NaN,...,1,1946-07-21,1.0,145,NaN,NaN,NaN,145,5,18.1
1,200,Bolivia,Government of Bolivia,23,NaN,MNR,720,NaN,2,NaN,...,1,1952-04-12,1.0,145,NaN,NaN,NaN,145,5,18.1
2,200,Bolivia,Government of Bolivia,23,NaN,ELN,721,NaN,2,NaN,...,1,1967-10-16,1.0,145,NaN,NaN,NaN,145,5,18.1
3,201,Cambodia (Kampuchea),Government of France,33,NaN,Khmer Issarak,160,NaN,1,Cambodia,...,0,NaN,NaN,220,NaN,NaN,NaN,811,3,18.1
4,201,Cambodia (Kampuchea),Government of France,33,NaN,Khmer Issarak,160,NaN,1,Cambodia,...,0,NaN,NaN,220,NaN,NaN,NaN,811,3,18.1


In [56]:
def integerize(x):
    try:
        return int(x)
    except ValueError:
        return 0
    

country_conf = conf[conf.gwno_b.apply(integerize) > 0]
print(country_conf.shape)
print(country_conf.columns)
country_conf.head()


(125, 28)
Index(['conflict_id', 'location', 'side_a', 'side_a_id', 'side_a_2nd',
       'side_b', 'side_b_id', 'side_b_2nd', 'incompatibility',
       'territory_name', 'year', 'intensity_level', 'cumulative_intensity',
       'type_of_conflict', 'start_date', 'start_prec', 'start_date2',
       'start_prec2', 'ep_end', 'ep_end_date', 'ep_end_prec', 'gwno_a',
       'gwno_a_2nd', 'gwno_b', 'gwno_b.1', 'gwno_loc', 'region', 'version'],
      dtype='object')


,conflict_id,location,side_a,side_a_id,side_a_2nd,side_b,side_b_id,side_b_2nd,incompatibility,territory_name,...,ep_end,ep_end_date,ep_end_prec,gwno_a,gwno_a_2nd,gwno_b,gwno_b.1,gwno_loc,region,version
119,214,"France, Thailand",Government of France,33,NaN,Government of Thailand,147,NaN,1,Northern Cambodia,...,1,1946-11-17,1.0,220,NaN,800,NaN,"220, 800","1, 3",18.1
120,215,"Albania, United Kingdom",Government of Albania,45,NaN,Government of United Kingdom,28,NaN,1,Korfu Channel,...,1,1946-12-31,5.0,339,NaN,200,NaN,"200, 339",1,18.1
131,218,"India, Pakistan",Government of India,141,NaN,Government of Pakistan,142,NaN,1,Kashmir,...,1,1948-12-31,1.0,750,NaN,770,NaN,"750, 770",3,18.1
132,218,"India, Pakistan",Government of India,141,NaN,Government of Pakistan,142,NaN,1,Kashmir,...,0,NaN,NaN,750,NaN,770,NaN,"750, 770",3,18.1
133,218,"India, Pakistan",Government of India,141,NaN,Government of Pakistan,142,NaN,1,Kashmir,...,1,1965-12-15,1.0,750,NaN,770,NaN,"750, 770",3,18.1


In [25]:
print(country_conf['gwno_a'])

119                         220
120                         339
131                         750
132                         750
133                         750
134                         750
135                         750
136                         750
137                         750
138                         750
139                         750
140                         750
141                         750
142                         750
143                         750
144                         750
145                         750
146                         750
147                         750
148                         750
149                         750
150                         750
151                         750
152                         750
314                         751
348     651, 645, 663, 660, 652
349     651, 645, 663, 660, 652
424                         710
425                         710
426                         710
                 ...           
1567    

In [51]:
nodes = set()
country_name_gwno_mapping = {}


for i, row in country_conf.iterrows():
    for num, name in zip(row['gwno_a'].split(','), row['side_a'].split(',')):
        if num not in country_name_gwno_mapping:
            country_name_gwno_mapping[num]= name.strip()
        nodes.add(num.strip())
        
    for num, name in zip(row['gwno_b'].split(','), row['side_b'].split(',')):
        if num not in country_name_gwno_mapping:
            country_name_gwno_mapping[num]= name.strip()
        nodes.add(num.strip())

In [52]:
print(nodes)
# print(country_name_gwno_mapping)
for k, v in country_name_gwno_mapping.items():
    print(k, v)

{'160', '850', '93', '483', '751', '690', '750', '530', '95', '200', '775', '439', '620', '475', '130', '310', '432', '900', '626', '811', '812', '339', '92', '678', '616', '732', '640', '713', '645', '531', '220', '651', '663', '816', '770', '522', '731', '210', '710', '352', '500', '625', '652', '615', '700', '820', '91', '55', '520', '2', '666', '365', '680', '660', '135', '471', '510', '600', '800', '817', '630'}
220 Government of France
800 Government of Thailand
339 Government of Albania
200 Government of United Kingdom
750 Government of India
770 Government of Pakistan
751 Government of Hyderabad
651 Government of Egypt
 645 Government of Iraq
 663 Government of Jordan
 660 Government of Lebanon
 652 Government of Syria
666 Government of Israel
710 Government of China
713 Government of Taiwan
731 Government of North Korea
732 Government of South Korea
310 Government of Hungary
365 Government of Russia (Soviet Union)
91 Government of Honduras
93 Government of Nicaragua
530 Govern

In [36]:
links = {}

for i, row in country_conf.iterrows():
    side_a = [num.strip() for num in row['gwno_a'].split(',')]
    
    side_b = [num.strip() for num in row['gwno_b'].split(',')]
    
    for num_a in side_a:
        if num_a not in links:
            links[num_a] = {}
        for num_b in side_b:
            if num_b in links[num_a]:
                links[num_a][num_b] += 1
#                 links[num_a][num_b][1].append(row['start_date'])
            else:
                links[num_a][num_b] = 1
#                 links[num_a][num_b] = [1, [row['start_date']]]
                
    for num_b in side_b:
        if num_b not in links:
            links[num_b] = {}
        for num_a in side_a:
            if num_a in links[num_b]:
                links[num_b][num_a] += 1
            else:
                links[num_b][num_a] = 1
    

In [35]:
print(links)

{'220': {'800': 1, '616': 1}, '339': {'200': 1}, '750': {'770': 22}, '751': {'750': 1}, '651': {'666': 6, '200': 2}, ' 645': {'666': 2}, ' 663': {'666': 2}, ' 660': {'666': 2}, ' 652': {'666': 2}, '710': {'713': 4, '750': 2, '775': 1, '365': 1, '816': 10}, '731': {'732': 5}, '310': {'365': 1}, '91': {'93': 1}, '530': {'520': 4}, '850': {'210': 1, '820': 4}, '615': {'600': 1}, '817': {'816': 11}, '811': {'800': 3, '816': 3}, '666': {'663': 1, '652': 2}, '92': {'91': 1}, '680': {'678': 2}, '352': {'640': 1}, '630': {'645': 10}, '160': {'200': 1}, '483': {'475': 1, '620': 1}, '55': {'2': 1}, '439': {'432': 1}, '812': {'800': 3}, '95': {'2': 1}, '645': {'690': 2}, '130': {'135': 1}, '471': {'475': 1}, '531': {'530': 4}, '900': {'645': 1}, ' 200': {'645': 1}, ' 2': {'645': 1}, '510': {'500': 1}, '700': {'365': 1}, '522': {'531': 1}, '626': {'625': 1}}


In [38]:
nodes_json = [{'id': i}for i in nodes]
print(nodes_json)
links_json = [{'start': a, 'end': b, 'weight': w } for a, v in links.items() for b, w in v.items() ]
print(links_json)

[{'id': '160'}, {'id': '850'}, {'id': '93'}, {'id': '483'}, {'id': '751'}, {'id': '690'}, {'id': '750'}, {'id': '530'}, {'id': '95'}, {'id': '200'}, {'id': '775'}, {'id': '439'}, {'id': '620'}, {'id': '475'}, {'id': '130'}, {'id': ' 652'}, {'id': ' 660'}, {'id': '310'}, {'id': '432'}, {'id': '900'}, {'id': '626'}, {'id': ' 200'}, {'id': '811'}, {'id': '812'}, {'id': '339'}, {'id': '92'}, {'id': '678'}, {'id': '616'}, {'id': ' 2'}, {'id': '732'}, {'id': '640'}, {'id': '713'}, {'id': '645'}, {'id': '531'}, {'id': '220'}, {'id': '651'}, {'id': '663'}, {'id': '816'}, {'id': '770'}, {'id': '522'}, {'id': '731'}, {'id': '210'}, {'id': '710'}, {'id': '352'}, {'id': '500'}, {'id': '625'}, {'id': '615'}, {'id': ' 663'}, {'id': '652'}, {'id': '820'}, {'id': '700'}, {'id': '91'}, {'id': '55'}, {'id': '520'}, {'id': '2'}, {'id': '666'}, {'id': '365'}, {'id': '680'}, {'id': '135'}, {'id': ' 645'}, {'id': '471'}, {'id': '510'}, {'id': '600'}, {'id': '800'}, {'id': '817'}, {'id': '630'}]
[{'start': '

In [59]:
ccodes = pd.read_csv('COW country codes.csv')
ccodes.head()

,StateAbb,CCode,StateNme
0,USA,2,United States of America
1,CAN,20,Canada
2,BHM,31,Bahamas
3,CUB,40,Cuba
4,CUB,40,Cuba


In [60]:
ccode_converter = {}

for i, row in ccodes.iterrows():
    if row['CCode'] not in ccode_converter:
        ccode_converter[row['CCode']]=row['StateAbb']
    
print(ccode_converter)

{2: 'USA', 20: 'CAN', 31: 'BHM', 40: 'CUB', 41: 'HAI', 42: 'DOM', 51: 'JAM', 52: 'TRI', 53: 'BAR', 54: 'DMA', 55: 'GRN', 56: 'SLU', 57: 'SVG', 58: 'AAB', 60: 'SKN', 70: 'MEX', 80: 'BLZ', 90: 'GUA', 91: 'HON', 92: 'SAL', 93: 'NIC', 94: 'COS', 95: 'PAN', 100: 'COL', 101: 'VEN', 110: 'GUY', 115: 'SUR', 130: 'ECU', 135: 'PER', 140: 'BRA', 145: 'BOL', 150: 'PAR', 155: 'CHL', 160: 'ARG', 165: 'URU', 200: 'UKG', 205: 'IRE', 210: 'NTH', 211: 'BEL', 212: 'LUX', 220: 'FRN', 221: 'MNC', 223: 'LIE', 225: 'SWZ', 230: 'SPN', 232: 'AND', 235: 'POR', 240: 'HAN', 245: 'BAV', 255: 'GMY', 260: 'GFR', 265: 'GDR', 267: 'BAD', 269: 'SAX', 271: 'WRT', 273: 'HSE', 275: 'HSG', 280: 'MEC', 290: 'POL', 300: 'AUH', 305: 'AUS', 310: 'HUN', 315: 'CZE', 316: 'CZR', 317: 'SLO', 325: 'ITA', 327: 'PAP', 329: 'SIC', 331: 'SNM', 332: 'MOD', 335: 'PMA', 337: 'TUS', 338: 'MLT', 339: 'ALB', 341: 'MNG', 343: 'MAC', 344: 'CRO', 345: 'YUG', 346: 'BOS', 347: 'KOS', 349: 'SLV', 350: 'GRC', 352: 'CYP', 355: 'BUL', 359: 'MLD', 360